In [13]:
import sqlite3
import time

conn_index = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/dbv3-index.db")
cur_index = conn_index.cursor()
conn_core = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/dbv3-core.db")
cur_core = conn_core.cursor()
sec_conn_core = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/dbv3-core.db")
sec_cur_core = sec_conn_core.cursor()
thr_conn_core = sqlite3.connect("../feature_machine_learning_test/bitcoin_db/dbv3-core.db")
thr_cur_core = thr_conn_core.cursor()

In [2]:
from secret import rpc_user, rpc_password

In [4]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

Bitcoind PID: 18685



In [5]:
from address_convertor import pubkey_to_address, get_pubkey

In [6]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [7]:
import datetime
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

Best Block Heights: 641535, Time: 2020-07-31T10:13:09+09:00


In [82]:
import pickle

category_clusters = dict()
with open('./1_create_multiinpu_cluster_file/multiinput_cluster.pickle', 'rb') as fr:
    category_clusters = pickle.load(fr)

## 특징 연산 코드

In [118]:
# preparation database
import sqlite3

path = 'addr_feature.db'
conn_feature = sqlite3.connect(path)
cur = conn_feature.cursor()

def insert_feature(addr_id, rbtc, sbtc, age, balance, fee):
    cur.execute('''INSERT OR IGNORE INTO AddrFeat (
                       addr, received_btc, spent_btc, age, balance, fee) VALUES (
                       ?, ?, ?, ?, ?, ?);
                    ''', (addr_id, rbtc, sbtc, age, balance, fee,))
    
def check_exist_data(addr_id):
    cur.execute('''SELECT EXISTS ( SELECT * FROM AddrFeat WHERE addr = ? )''', (addr_id,))
    return cur.fetchone()[0]
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def journal_mode(mode):
    sql = f'PRAGMA journal_mode = {mode}'
    cur.execute(sql)
    conn.commit()

def synchronous(mode):
    sql = f'PRAGMA synchronous = {mode}'
    cur.execute(sql)
    conn.commit()

In [12]:
cur_core.execute('''SELECT sql FROM sqlite_master WHERE name="TxIn"''')
print(cur_core.fetchone()[0])
cur_core.execute('''SELECT sql FROM sqlite_master WHERE name="TxOut"''')
print(cur_core.fetchone()[0])

CREATE TABLE TxIn (
      tx INTEGER NOT NULL,
      n INTEGER NOT NULL,
      ptx INTEGER NOT NULL,
      pn INTEGER NOT NULL,
      UNIQUE (tx, n))
CREATE TABLE TxOut (
      tx INTEGER NOT NULL,
      n INTEGER NOT NULL,
      addr INTEGER NOT NULL,
      btc REAL NOT NULL,
      UNIQUE (tx, n, addr))


In [15]:
# bitcoin 시세 (2010-01-01 ~ 2020-03-28)
# https://kr.investing.com/crypto/bitcoin/historical-data
import pandas as pd
from datetime import datetime
bitcoin_past_csv = pd.read_csv('./bitcoin_past_value.csv')

def change_date_format(date):
    year = date.split('년 ')
    month = year[1].split('월 ')
    day = month[1].split('일')[0]
    
    return year[0] + '-' + month[0] + '-' + day

bitcoin_past_csv = bitcoin_past_csv[['날짜', '종가']]
bitcoin_past_csv['date'] = bitcoin_past_csv.apply(lambda x:change_date_format(x['날짜']), axis=1)

preprocess_csv = bitcoin_past_csv.copy()
preprocess_csv = preprocess_csv.drop(['날짜'], axis=1)
preprocess_csv = preprocess_csv.rename(columns = {'종가' : 'USD'})
preprocess_csv = preprocess_csv[['date', 'USD']]
preprocess_csv.head()

,date,USD
0,2020-05-22,"9,066.3"
1,2020-05-21,"9,059.0"
2,2020-05-20,"9,512.3"
3,2020-05-19,"9,773.3"
4,2020-05-18,"9,730.7"


In [131]:
def get_addr_btc_usd_spent(addr_id):
    total_spent_value = 0.0; total_spent_usd = 0.0; convert_susd = 0.0
    cur_core.execute('''SELECT tx, SUM(btc) FROM TxOut WHERE addr = ? GROUP BY tx''', (addr_id,))
    for tx, btc in cur_core:
        # btc 계산
        total_spent_value += btc
        
        # usd 계산
        sec_cur_core.execute('''SELECT BlkTime.unixtime FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk
                                WHERE BlkTx.tx = ?''', (tx,))
        get_time = sec_cur_core.fetchone()[0]
        itime = datetime.fromtimestamp(get_time).strftime('%Y-%m-%d') # convert to datetime
        try:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == itime]['USD'])[0].replace(",","")) # search usd
        except IndexError:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == '2010-07-18']['USD'])[0].replace(",","")) # search usd
        total_spent_usd += btc * convert_susd
        
#     btc_spent.appent(total_spent_value)
#     usd_apent.append(total_spent_usd)
    return total_spent_value, total_spent_usd

def get_addr_btc_usd_received(addr_id):
    total_received_value = 0.0 # btc
    total_received_usd = 0.0; convert_susd = 0.0 # usd
    sum_utxo_btc = 0.0 # balance
    
    cur_core.execute('''SELECT TxIn.tx, SUM(TxOut.btc) FROM TxIn INNER JOIN TxOut 
                            ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n WHERE TxOut.addr = ? GROUP BY TxIn.tx''', (addr_id,))
    for tx, btc in cur_core:
        # btc 계산
        total_received_value += btc
        
        # usd 계산
        sec_cur_core.execute('''SELECT BlkTime.unixtime FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk
                                WHERE BlkTx.tx = ?''', (tx,))
        get_time = sec_cur_core.fetchone()[0]
        itime = datetime.fromtimestamp(get_time).strftime('%Y-%m-%d') # convert to datetime
        try:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == itime]['USD'])[0].replace(",","")) # search usd
        except IndexError:
            convert_susd = float(list(preprocess_csv[preprocess_csv['date'] == '2010-07-18']['USD'])[0].replace(",","")) # search usd

        total_received_usd += btc * convert_susd
        
        # balance 계산
        sec_cur_core.execute('''SELECT addr, SUM(btc) FROM TxOut WHERE tx = ? GROUP BY addr''', (tx,))
        for addr, btc in sec_cur_core:
            try:
                thr_cur_core.execute('''SELECT EXISTS ( SELECT * FROM TxIn INNER JOIN TxOut 
                                            ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n WHERE TxOut.addr = ? )''', (addr,))
                if thr_cur_core.fetchone()[0] == 0:
                    sum_utxo_btc += btc
            except KeyError:
                continue
    
#     btc_received.append(total_received_value)
#     usd_received.append(total_received_usd)
#     balance.append(sum_utxo_btc)
    return total_received_value, total_received_usd, sum_utxo_btc

def get_age(addr_id):
    txhash_list = list()
    
    # 처음 생성은 tx_out 가장 첫번째에서...
    cur_core.execute('''SELECT MIN(BlkTime.unixtime) FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk
                            JOIN TxOut ON BlkTx.tx = TxOut.tx WHERE TxOut.addr = ?''', (addr_id,))
    txhash_list.append(cur_core.fetchone()[0])
    
    # 마지막 생성은 tx_in, tx_out 내림차순으로 정리해서 확인
    cur_core.execute('''SELECT MAX(BlkTime.unixtime) FROM (SELECT TxIn.tx FROM TxIn INNER JOIN TxOut 
                            ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n WHERE TxOut.addr = ?) AS T
                            INNER JOIN BlkTx ON T.tx = BlkTx.tx 
                            INNER JOIN BlkTime ON BlkTx.blk = BlkTime.blk''', (addr_id,))
    txin_txhash = cur_core.fetchone()[0]
    if txin_txhash == None:
        txin_txhash = 0
        
    cur_core.execute('''SELECT MAX(BlkTime.unixtime) FROM BlkTime JOIN BlkTx ON BlkTime.blk = BlkTx.blk 
                    JOIN TxOut ON BlkTx.tx = TxOut.tx WHERE TxOut.addr = ?''', (addr_id,))
    txhash_list.append([txin_txhash, cur_core.fetchone()[0]])
    
    try:
        first_check_time = 0; last_check_time = 0
        for i, time in enumerate(txhash_list):
            if i == 0: # 처음 생성된 시간을 구하는.
                first_check_time = time
            else: # 마지막으로 사용된 시간을 구하는.
                for th_time in time:
                    if last_check_time < th_time:
                        last_check_time = th_time
                    
    
#     age.append(last_check_time - first_check_time)
        return last_check_time - first_check_time
    except:
        return 0

def get_fee(addr_id):
    activate_txin = set(); activate_txout = set(); fee_list = list()
    cur_core.execute('''SELECT SUM(btc) FROM TxOut WHERE addr = ?''', (addr_id,))
    txout_value = cur_core.fetchone()[0]
    
    cur_core.execute('''SELECT SUM(TxOut.btc) FROM TxIn INNER JOIN TxOut ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n 
                            WHERE TxOut.addr = ?''', (addr_id,))
    txin_value = cur_core.fetchone()[0]
    if txin_value == None:
        txin_value = 0.0
            
#     fee.append(txin_value - txout_value)
    return txin_value - txout_value

# def get_number_spent_received(addr_id, number_spent, number_received):
#     cur_core.execute('''SELECT COUNT(*) FROM TxOut WHERE addr = ?''', (addr_id,))
#     number_spent.append(cur_core.fetchone()[0])
    
#     cur_core.execute('''SELECT COUNT(*) FROM TxIn INNER JOIN TxOut ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n 
#                                 WHERE TxOut.addr = ?''', (addr_id,))
#     number_received.append(cur_core.fetchone()[0])
    
def get_number_spent(addr_id):
    cur_core.execute('''SELECT COUNT(*) FROM TxOut WHERE addr = ?''', (addr_id,))
    return cur_core.fetchone()[0]

def get_number_received(addr_id):
    cur_core.execute('''SELECT COUNT(*) FROM TxIn INNER JOIN TxOut ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n 
                                WHERE TxOut.addr = ?''', (addr_id,))
    return cur_core.fetchone()[0]

In [117]:
import time
import numpy as np
import statistics

def median_avg_std(flist):
#     try:
    return (statistics.median(flist), np.mean(flist), np.std(flist))
#     except:
#         print(flist)

def control_address_feature(cluster_addresses):
    btc_spent_list = list(); btc_received_list = list()
    usd_spent_list = list(); usd_received_list = list()
    balance_list = list(); age_list = list()
    fee_tx_list = list(); number_spent_list = list(); number_received_list = list()
    
    for find_addr_id in tqdm(cluster_addresses):
        spent_btc, spent_usd = get_addr_btc_usd_spent(find_addr_id)
        btc_spent_list.append(spent_btc)
        usd_spent_list.append(spent_usd)
    
        received_btc, received_usd, balance = get_addr_btc_usd_received(find_addr_id)
        btc_received_list.append(received_btc)
        usd_received_list.append(received_usd)
        balance_list.append(balance)
        
        age = get_age(find_addr_id)
        age_list.append(age)
        
        fee = get_fee(find_addr_id)
        fee_tx_list.append(fee)
    
        number_spent_list.append(get_number_spent(find_addr_id))
        number_received_list.append(get_number_received(find_addr_id))
        
        if check_exist_data(find_addr_id) == 0:
            insert_feature(find_addr_id, spent_btc, spent_btc, age, balance, fee)
    
    return [median_avg_std(btc_spent_list), median_avg_std(btc_received_list), median_avg_std(usd_spent_list), 
    median_avg_std(usd_received_list), median_avg_std(balance_list), median_avg_std(age_list),
    median_avg_std(fee_tx_list), median_avg_std(number_spent_list), median_avg_std(number_received_list)]

In [137]:
import time
import numpy as np
import pickle
from tqdm import tqdm

rpc_connection = get_rpc()
# cluster_feature_exchange = dict()
total_time = time.time()
try:
    begin_transactions()
    for cc in ['ransomware28']:
        cluster_feature_exchange[cc] = control_address_feature(category_clusters[cc])
    commit_transactions()
except Exception as e:
    print(e)
    commit_transactions()
finally:
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
    with open('./2_create_feature_extract_file/feature_all_category.pickle', 'wb') as f:
        pickle.dump(cluster_feature_exchange, f)
    conn_feature.close()
    
print('total_time : ' + str(time.time()-total_time))

100%|██████████| 5319/5319 [00:17<00:00, 303.66it/s]

total_time : 17.57232928276062
